<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=46c2281699687425ab8dab601cc23a0a454d6bc4dbc3323e722d2826494748bb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 10:41:47
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 40.63 K (0.29%)
Current PnL: -21.58 L (-14.38%)
CY Booked + Current PnL: -9.68 L (-6.45%)
-------------------
Total profit:  1.68 L
Total loss:  -23.26 L
-------------------
Total Booked + Current PnL: 17.30 L (14.05%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.51%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.77 L (63.45%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.44,58.0,X-LC,3.78,214610.0,10110.0,13177.0,0.50,4.94,6.14,11.39,10.0,0.77,1.55,14.34,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,95.73,47.0,M-SC,1.67,84344.0,-16433.0,16523.0,0.06,-16.31,19.59,0.09,245.0,-0.99,0.61,10.81,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.02,58.0,X-MC,1.26,212040.0,20298.0,17451.0,0.11,10.59,8.23,19.69,99.0,1.16,1.53,18.10,XY25,NTT,AC
33,ICICIGI,2260.25,-13.14,68.0,X-MC,2.10,149547.0,13414.0,17706.0,0.03,9.85,11.84,22.86,91.0,0.76,1.08,23.71,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.85,215124.0,4758.0,24029.0,0.34,2.26,11.17,13.68,37.0,0.20,1.55,19.16,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.0,1196.65,57.0,M-SC,23.05,176265.0,-4878.0,50341.0,3.88,-2.69,28.56,25.10,235.0,-0.10,1.27,41.26,XY24,NTT,HEALTHCARE
66,SONACOMS,1006.0,-34.05,69.0,M-SC,12.29,82049.0,-19110.0,94004.0,3.67,-18.89,114.57,74.03,202.0,-0.20,0.59,15.70,AR,BTT,AUTO
84,WHIRLPOOL,2270.0,-45.59,49.0,M-SC,4.26,91875.0,376.0,78379.0,3.55,0.41,85.31,86.07,223.0,0.00,0.66,33.42,XR,NTT,DURABLES
24,EASEMYTRIP,26.4,-18.06,42.0,M-SC,25.55,78845.0,-97970.0,178765.0,2.93,-55.41,226.73,45.70,197.0,-0.55,0.57,2.93,XY24,NTT,TRAVEL
4,ANGELONE,3033.0,15.82,70.0,X-SC,9.34,205262.0,14256.0,43433.0,2.37,7.46,21.16,30.21,157.0,0.33,1.48,29.90,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,856.52,36.0,H-SC,5.20,82260.0,-26460.0,106140.0,-1.79,-24.34,129.03,73.29,161.0,-0.25,0.59,10.03,XR,NTT,ENTERTAINMENT
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.34,141879.0,-134644.0,244713.0,-1.16,-48.69,172.48,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
10,BANDHANBNK,400.00,-13.31,46.0,H-SC,2.90,216222.0,-62337.0,319771.0,-1.13,-22.38,147.89,92.42,151.0,-0.19,1.56,24.96,XY24,NTT,BANKS
15,CAMPUS,393.00,-23.07,58.0,M-SC,2.79,160316.0,-7459.0,63309.0,-1.05,-4.45,39.49,33.28,210.0,-0.12,1.15,24.26,XY24,NTT,FOOTWEAR
50,PGHH,17907.65,-26.84,57.0,X-MC,3.25,211935.0,11115.0,56671.0,-1.04,5.53,26.74,33.76,80.0,0.20,1.53,11.02,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-45.59,49.0,M-SC,4.26,91875.0,376.0,78379.0,3.55,0.41,85.31,86.07,223.0,0.00,0.66,33.42,XR,NTT,DURABLES
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.50,149022.0,-198.0,144373.0,0.36,-0.13,96.88,96.62,208.0,-0.00,1.07,61.27,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,49.0,M-MC,12.22,226133.0,1171.0,166479.0,0.79,0.52,73.62,74.53,192.0,0.01,1.63,32.94,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.57,38.0,H-SC,8.06,124068.0,732.0,130892.0,0.48,0.59,105.50,106.72,119.0,0.01,0.89,24.11,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,11.90,131193.0,-11286.0,30804.0,0.04,-7.92,23.48,13.70,163.0,-0.37,0.95,54.84,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,95.73,47.0,M-SC,1.67,84344.0,-16433.0,16523.0,0.06,-16.31,19.59,0.09,245.0,-0.99,0.61,10.81,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,52.0,H-SC,1.77,222462.0,-47205.0,83534.0,0.62,-17.50,37.55,13.47,138.0,-0.57,1.60,13.12,XY24,NTT,PAINTS
17,CERA,9475.0,-23.39,43.0,H-SC,2.19,139668.0,-36235.0,78256.0,0.32,-20.60,56.03,23.89,149.0,-0.46,1.01,20.62,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.21,46.0,H-MC,7.15,102572.0,-29263.0,71493.0,0.58,-22.20,69.70,32.03,98.0,-0.41,0.74,14.89,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.14,207872.0,4702.0,37438.0,0.57,2.31,18.01,20.74,55.0,0.13,1.50,7.99,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,49.0,M-MC,12.22,226133.0,1171.0,166479.0,0.79,0.52,73.62,74.53,192.0,0.01,1.63,32.94,XY24,BTT,STEEL
31,HINDZINC,730.22,36.19,68.0,M-LC,9.51,222328.0,17252.0,95312.0,-0.41,8.41,42.87,54.89,52.0,0.18,1.60,34.54,X5K,ATH,METALS
85,WIPRO,420.00,-11.98,54.0,M-LC,5.86,154578.0,3633.0,105824.0,-0.36,2.41,68.46,72.51,53.0,0.03,1.11,8.57,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,55.0,M-LC,8.74,157941.0,17101.0,27387.0,-0.38,12.14,17.34,31.59,66.0,0.62,1.14,41.85,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,68.0,M-LC,9.51,222328.0,17252.0,95312.0,-0.41,8.41,42.87,54.89,52.0,0.18,1.60,34.54,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,13.01,193204.0,13396.0,99017.0,-0.57,7.45,51.25,62.52,88.0,0.14,1.39,44.08,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,54.0,M-LC,5.86,154578.0,3633.0,105824.0,-0.36,2.41,68.46,72.51,53.0,0.03,1.11,8.57,XR,NTT,IT
36,INDIAMART,4810.62,-55.57,38.0,H-SC,8.06,124068.0,732.0,130892.0,0.48,0.59,105.50,106.72,119.0,0.01,0.89,24.11,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-45.59,49.0,M-SC,4.26,91875.0,376.0,78379.0,3.55,0.41,85.31,86.07,223.0,0.00,0.66,33.42,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.34,141879.0,-134644.0,244713.0,-1.16,-48.69,172.48,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.30,33.0,X-MC,1.88,227417.0,-20314.0,95379.0,-0.44,-8.20,41.94,30.30,92.0,-0.21,1.64,4.00,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52199.0,-12807.0,163738.0,0.02,-19.70,313.68,232.18,198.0,-0.08,0.38,1.79,XY24,NTT,MISC
12,BATAINDIA,2096.00,-36.58,40.0,X-SC,4.33,94869.0,-33801.0,79102.0,1.72,-26.27,83.38,35.20,219.0,-0.43,0.68,8.89,X40,NTT,FOOTWEAR
34,ICICIPRULI,790.00,-22.83,40.0,X-MC,2.20,132718.0,-3070.0,45828.0,-0.13,-2.26,34.53,31.48,107.0,-0.07,0.96,9.58,X40,ATH,INSURANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.62,156240.0,-29855.0,77339.0,0.20,-16.04,49.50,25.51,15.0,-0.39,1.13,15.14,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.02,58.0,X-MC,1.26,212040.0,20298.0,17451.0,0.11,10.59,8.23,19.69,99.0,1.16,1.53,18.10,XY25,NTT,AC
28,HAVELLS,2069.16,-8.30,33.0,X-MC,1.88,227417.0,-20314.0,95379.0,-0.44,-8.20,41.94,30.30,92.0,-0.21,1.64,4.00,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.14,68.0,X-MC,2.10,149547.0,13414.0,17706.0,0.03,9.85,11.84,22.86,91.0,0.76,1.08,23.71,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-6.95,63.0,X-LC,2.39,286486.0,21060.0,35725.0,0.24,7.93,12.47,21.40,11.0,0.59,2.06,15.10,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.34,141879.0,-134644.0,244713.0,-1.16,-48.69,172.48,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52199.0,-12807.0,163738.0,0.02,-19.70,313.68,232.18,198.0,-0.08,0.38,1.79,XY24,NTT,MISC
73,TCS,4389.96,-30.61,43.0,X-LC,12.51,277027.0,-68929.0,135632.0,-0.76,-19.92,48.96,19.28,1.0,-0.51,2.00,2.41,X40,ATH,IT
41,ITC,452.00,-39.19,47.0,X-LC,2.68,196505.0,-3633.0,24524.0,0.49,-1.82,12.48,10.44,4.0,-0.15,1.42,3.65,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.48,44.0,X-MC,5.81,180318.0,-20188.0,117351.0,-0.16,-10.07,65.08,48.45,90.0,-0.17,1.30,3.98,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.52,52199.0,-12807.0,163738.0,0.02,-19.70,313.68,232.18,198.0,-0.08,0.38,1.79,XY24,NTT,MISC
55,RELAXO,1176.00,-39.59,43.0,X-SC,4.02,82391.0,-62769.0,138697.0,-0.85,-43.24,168.34,52.31,136.0,-0.45,0.59,10.28,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.58,40.0,X-SC,4.33,94869.0,-33801.0,79102.0,1.72,-26.27,83.38,35.20,219.0,-0.43,0.68,8.89,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,50.0,X-SC,1.01,107691.0,5687.0,38241.0,0.26,5.58,35.51,43.06,122.0,0.15,0.78,23.65,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.41,108135.0,-19803.0,66936.0,-0.30,-15.48,61.90,36.84,143.0,-0.30,0.78,10.70,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-30.61,43.0,X-LC,12.51,277027.0,-68929.0,135632.0,-0.76,-19.92,48.96,19.28,1.0,-0.51,2.00,2.41,X40,ATH,IT
39,INFY,2275.00,-19.96,45.0,X-LC,7.55,311938.0,-950.0,172626.0,-0.67,-0.30,55.34,54.87,3.0,-0.01,2.25,6.33,X40,BTT,IT
41,ITC,452.00,-39.19,47.0,X-LC,2.68,196505.0,-3633.0,24524.0,0.49,-1.82,12.48,10.44,4.0,-0.15,1.42,3.65,X40,NTT,FMCG
82,VBL,671.64,-19.44,49.0,X-LC,10.65,290226.0,-25616.0,138293.0,2.14,-8.11,47.65,35.67,5.0,-0.19,2.09,4.62,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,47.0,H-LC,7.26,248280.0,-13339.0,132557.0,-0.02,-5.10,53.39,45.57,7.0,-0.10,1.79,5.65,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6588.89,51.0,L-SC,6.11,73396.0,-20414.0,97030.0,-0.32,-21.76,132.20,81.67,269.0,-0.21,0.53,42.13,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,53.0,L-SC,28.82,83309.0,-30240.0,70288.0,1.83,-26.63,84.37,35.27,268.0,-0.43,0.60,103.90,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,43.0,H-SC,9.47,90390.0,-7590.0,29196.0,0.07,-7.75,32.30,22.05,152.0,-0.26,0.65,31.39,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,11.90,131193.0,-11286.0,30804.0,0.04,-7.92,23.48,13.70,163.0,-0.37,0.95,54.84,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,10.29,145034.0,-34037.0,99769.0,0.08,-19.01,68.79,36.71,135.0,-0.34,1.04,43.90,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.50,149022.0,-198.0,144373.0,0.36,-0.13,96.88,96.62,208.0,-0.00,1.07,61.27,XY25,NTT,FINANCE
4,ANGELONE,3033.00,15.82,70.0,X-SC,9.34,205262.0,14256.0,43433.0,2.37,7.46,21.16,30.21,157.0,0.33,1.48,29.90,X40N,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.29,145034.0,-34037.0,99769.0,0.08,-19.01,68.79,36.71,135.0,-0.34,1.04,43.90,XR,NTT,BANKS
63,SHALBY,327.00,1196.65,57.0,M-SC,23.05,176265.0,-4878.0,50341.0,3.88,-2.69,28.56,25.10,235.0,-0.10,1.27,41.26,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,36.19,68.0,M-LC,9.51,222328.0,17252.0,95312.0,-0.41,8.41,42.87,54.89,52.0,0.18,1.60,34.54,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,43.98
2,50,75.85


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.48
LC    33.69
MC    21.86
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.64
X40      14.77
XY25     12.16
XR       10.96
X40N     10.24
AR        8.20
OX40N     7.67
X200      1.83
X5K       1.60
SR        1.14
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.57
X-LC    22.56
X-MC    16.11
M-SC    12.75
M-LC     5.35
H-LC     4.75
X-SC     4.69
H-MC     3.82
M-MC     1.63
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.09,-4.93,39.52
IT,12.95,-17.10,80.44
FINANCE,12.26,-11.11,61.48
MISC,6.89,-20.10,86.64
BANKS,6.38,-12.66,71.93
PAINTS,5.57,-19.44,37.24
ELECTRICAL,5.34,-11.26,50.29
INSURANCE,3.94,-1.26,39.11
AUTO,2.83,-46.13,115.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3321126.0,22
XR,1386093.0,14
AR,1192326.0,9
X40,791312.0,10
X40N,606510.0,8
OX40N,584607.0,10
XY25,493341.0,8
SR,264460.0,2
X5K,95312.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3453258.0,25
M-SC,1271412.0,15
X-LC,1108206.0,13
X-MC,1092288.0,11
X-SC,530147.0,6
H-MC,349754.0,3
H-LC,281086.0,3
M-LC,265961.0,4
L-SC,256822.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1198374.0      6
           XR         825322.0      7
           AR         821372.0      5
M-SC       XY24       686944.0      6
X-MC       XY24       589354.0      4
X-LC       X40        496626.0      5
           XY24       309606.0      2
X-SC       X40N       287307.0      4
H-SC       OX40N      272775.0      4
           SR         264460.0      2
X-MC       X40        215584.0      4
H-LC       AR         209896.0      2
X-LC       X40N       201852.0      3
H-MC       XY24       179244.0      1
X-MC       XY25       169999.0      2
L-SC       XR         167318.0      2
M-MC       XY24       166479.0      1
X-SC       XY24       163738.0      1
M-SC       AR         161058.0      2
           OX40N      150835.0      4
           XY25       144373.0      1
           XR         128202.0      2
X-MC       X40N       117351.0      1
M-LC       XR         105824.0      1
X-LC       XY25       100122.0      3
H-MC       XR          99017.0      1
M-LC       X5K         95312.0      1
L-SC       OX40N       89504.0      1
X-SC       X40         79102.0      1
H-MC       OX40N       71493.0      1
H-LC       X200        71190.0      1
H-SC       MH          70955.0      1
L-MC       XR          60410.0      1
L-LC       XY25        41409.0      1
M-LC       XY25        37438.0      1
           XY24        27387.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
